In [1]:
%load_ext autoreload

%autoreload 2
import time
import numpy as np
import networkx as nx
import pymde

import numpy as np
import torch

from fury.stream.widget import Widget

from helios import NetworkDraw


### 1 - create a SBM graph and extract the edges list


In [2]:
size = 100
s = size
sizes = [s, s, s]
probs = np.array([[0.45, 0.05, 0.02], [0.05, 0.45, 0.07], [0.02, 0.07, 0.40]])/10
g = nx.stochastic_block_model(sizes, probs, seed=0)
'''
pin = .05
pout = .1
probs = [[pin, pout, 0], [0.0, pin, pout], [pout, 0.0, pin]]
#probs = [[pin, pout, 0], [0.0, pin, pout], [pout, 0.0, pin]]

g = nx.stochastic_block_model(sizes, probs, seed=0, directed=True)
'''
num_nodes = len(g)
edges_list = []
for source, target in g.edges():
    edges_list.append([source, target])
edges_list = np.array(edges_list)


In [3]:
edges_torch = torch.tensor(edges_list)
weights_torch = torch.ones(edges_torch.shape[0])
g_torch = pymde.Graph.from_edges(edges_torch, weights_torch)

### node/edge colors and shape

In [4]:
# update edge colors
colors_by_block = [[1, 0, 0], [0, 1, 0,], [0, 0, 1]]
#colors_by_block = [[1, 0, 0, 240], [0, 1, 0, 240], [0, 0, 1, 0]]

edge_colors = []
for source, target in g.edges():
    c0 = colors_by_block[source//s]
    c1 = colors_by_block[target//s]
    edge_colors += [c0, c1]
    
colors_by_block = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
colors = np.array(
    [colors_by_block[i//s] 
     for i in range(len(g))]).astype('float64')
markers = [['o', 's', 'd'][i//s]  for i in range(len(g))]

edge_colors = np.array(edge_colors).astype('float64')

### 2 - Creates a network superactor and adds to a FURY Scene

In [5]:



    
centers = np.random.normal(size=(len(g), 2))
node_edge_opacity = .8
node_edge_color = [1, 1, 1]

node_edge_width = node_edge_opacity
node_opacity = node_edge_opacity
network_draw = NetworkDraw(
        positions=centers,
        colors=colors,
        scales=.2,
        node_edge_width=.2,
        node_edge_opacity=node_edge_opacity,
        node_opacity=node_opacity,
        node_edge_color=node_edge_color,
        marker=markers,
        edge_line_color=edge_colors,
        edge_line_width=1,
        edges=edges_list
)



### 4- Starts the streaming

In [6]:
from fury.stream.widget import Widget
#widget = Widget(showm, encoding='webrtc', port=8777)
widget = Widget(network_draw.showm)
widget.start()
time.sleep(2)

url: http://localhost:7084?iframe=1&encoding=mjpeg


In [7]:

widget.return_iframe(300)

### 5 - Minimum distortion embedding



In [8]:
shortest_paths_graph = pymde.preprocess.graph.shortest_paths(g_torch)

In [9]:
mde = pymde.MDE(
    len(g),
    2,
    shortest_paths_graph.edges,
    pymde.losses.WeightedQuadratic(shortest_paths_graph.distances))


In [10]:
embedding = mde.embed()

In [11]:
pos = embedding.cpu().numpy().astype('float64')



In [12]:
network_draw.positions = pos
network_draw.Render()

In [20]:
widget.stop()

# All-connected and cubic distortion function 2d plot

In [13]:
n_items = 20
edges = pymde.all_edges(n_items)

In [14]:
centers = np.random.normal(size=(n_items, 2))
from helios import NetworkDraw

network2 = NetworkDraw(
        positions=centers, 
        scales=.2,
        node_edge_width=.2,
        edge_line_color=(0, 0, 0),
        marker='3d',
        edges=edges.cpu().numpy()
)


In [15]:
network2.nodes.edge_width = 0

In [16]:
widget2 = Widget(network2.showm)
widget2.start()
time.sleep(3)

url: http://localhost:7083?iframe=1&encoding=mjpeg


In [17]:
widget2.return_iframe(300)

In [18]:
mde = pymde.MDE(
    n_items,
    embedding_dim=2,
    edges=edges,
    distortion_function=pymde.penalties.Cubic(weights=1.0),
    constraint=pymde.Standardized())
embedding = mde.embed()


In [19]:
pos2d = embedding.cpu().numpy().astype('float64')

In [20]:
network2.positions = pos2d
network2.Render()